In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/ultralytics/ultralytics
!pip install ultralytics

Cloning into 'ultralytics'...
remote: Enumerating objects: 22842, done.
remote: Counting objects: 100% (768/768), done.
remote: Compressing objects: 100% (418/418), done.
remote: Total 22842 (delta 488), reused 567 (delta 349), pack-reused 22074
Receiving objects: 100% (22842/22842), 13.65 MiB | 18.71 MiB/s, done.
Resolving deltas: 100% (15905/15905), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 9.9 MB/s eta 0:00:00


In [7]:
# input sample asset

import gdown

download_link = 'https://drive.google.com/uc?id=1qQkCDS7c6-PjecYuRF7yEuD_P4Vgwi3Z'
output_path = '/content/video-contoh-trim.mov'

gdown.download(download_link, output_path, quiet=False)
print('File berhasil diunduh dan disimpan di:', output_path)


Downloading...
From (original): https://drive.google.com/uc?id=1qQkCDS7c6-PjecYuRF7yEuD_P4Vgwi3Z
From (redirected): https://drive.google.com/uc?id=1qQkCDS7c6-PjecYuRF7yEuD_P4Vgwi3Z&confirm=t&uuid=62721889-17fd-4e9d-b312-bc379a151be9
To: /content/video-contoh-trim.mov
100%|██████████| 206M/206M [00:07<00:00, 26.4MB/s]

File berhasil diunduh dan disimpan di: /content/video-contoh-trim.mov


Ganti path detection model dengan penyimpanan best.pt

In [30]:
%%writefile settings.py
from pathlib import Path
import sys

file_path = Path(__file__).resolve()
root_path = file_path.parent

if root_path not in sys.path:
    sys.path.append(str(root_path))

ROOT = root_path.relative_to(Path.cwd())

# Sources
IMAGE = 'Image'
VIDEO = 'Video'

SOURCES_LIST = [IMAGE, VIDEO]

IMAGES_DIR = ROOT / 'images'

VIDEO_DIR = Path('/content')
VIDEO_1_PATH = VIDEO_DIR / 'video-contoh-trim.mov'

VIDEOS_DICT = {
    'video_1': VIDEO_1_PATH,

}

# Model config
MODEL_DIR = ROOT / 'weights'
DETECTION_MODEL = '/content/drive/MyDrive/Sistem_Pengenalan_Pelindung/dataset_new/runs/detect/yolov8n_v2_train/weights/best.pt'


Overwriting settings.py


In [31]:
%%writefile helper.py

from ultralytics import YOLO
import time
import streamlit as st
import cv2
import settings

def load_model(model_path):
    model = YOLO(model_path)
    return model


def display_tracker_options():
    display_tracker = st.radio("Display Tracker", ('Yes', 'No'))
    is_display_tracker = True if display_tracker == 'Yes' else False
    if is_display_tracker:
        tracker_type = st.radio("Tracker", ("bytetrack.yaml", "botsort.yaml"))
        return is_display_tracker, tracker_type
    return is_display_tracker, None


def _display_detected_frames(conf, model, st_frame, image, is_display_tracking=None, tracker=None):


    # Resize the image to a standard size
    image = cv2.resize(image, (720, int(720*(9/16))))

    # Display object tracking, if specified
    if is_display_tracking:
        res = model.track(image, conf=conf, persist=True, tracker=tracker)
    else:
        # Predict the objects in the image using the YOLOv8 model
        res = model.predict(image, conf=conf)

    # # Plot the detected objects on the video frame
    res_plotted = res[0].plot()
    st_frame.image(res_plotted,
                   caption='Detected Video',
                   channels="BGR",
                   use_column_width=True
                   )


def play_stored_video(conf, model):
    source_vid = st.sidebar.selectbox(
        "Choose a video...", settings.VIDEOS_DICT.keys())

    is_display_tracker, tracker = display_tracker_options()

    with open(settings.VIDEOS_DICT.get(source_vid), 'rb') as video_file:
        video_bytes = video_file.read()
    if video_bytes:
        st.video(video_bytes)

    if st.sidebar.button('Detect Video Objects'):
        try:
            vid_cap = cv2.VideoCapture(
                str(settings.VIDEOS_DICT.get(source_vid)))
            st_frame = st.empty()
            while (vid_cap.isOpened()):
                success, image = vid_cap.read()
                if success:
                    _display_detected_frames(conf,
                                             model,
                                             st_frame,
                                             image,
                                             is_display_tracker,
                                             tracker
                                             )
                else:
                    vid_cap.release()
                    break
        except Exception as e:
            st.sidebar.error("Error loading video: " + str(e))

Overwriting helper.py


In [32]:
%%writefile app.py
import streamlit as st

from pathlib import Path
import PIL

# Local Modules
import settings
import helper


st.set_page_config(
    page_title="Consctruction Safety Detection using YOLO",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Main page
st.header("Consctruction Safety Detection using YOLO", divider='rainbow')

# Sidebar
st.sidebar.header("Configuration")
confidence = float(st.sidebar.slider( "Select Model Confidence", 25, 100, 40)) / 100
model_path = Path(settings.DETECTION_MODEL)

# Load Pre-trained Model
try:
    model = helper.load_model(model_path)
except Exception as ex:
    st.error(f"Unable to load model. Check path: {model_path}")
    st.error(ex)

st.sidebar.header("Image/Video Config")
source_radio = st.sidebar.radio(
    "Select Source", settings.SOURCES_LIST)

source_img = None

# If image is selected
if source_radio == settings.IMAGE:
    source_img = st.sidebar.file_uploader(
        "Choose an image...", type=("jpg", "jpeg", "png", 'bmp', 'webp'))

    col1, col2 = st.columns(2)

    with col1:
        try:
            if source_img is None:
                st.image(st.image("https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon@2.png", width=400), caption="Default Image", use_column_width=True)
            else:
                uploaded_image = PIL.Image.open(source_img)
                st.image(source_img, caption="Uploaded Image",
                         use_column_width=True)
        except Exception as ex:
            st.error(ex)

    with col2:
        if source_img is None:
            st.subheader('Masukkan gambar atau video anda terlebih dahulu')
        else:
            if st.sidebar.button('Detect Objects'):
                res = model.predict(uploaded_image,
                                    conf=confidence
                                    )
                boxes = res[0].boxes
                res_plotted = res[0].plot()[:, :, ::-1]
                st.image(res_plotted, caption='Detected Image',
                         use_column_width=True)
                try:
                    with st.expander("Detection Results"):
                        for box in boxes:
                            st.write(box.data)
                except Exception as ex:
                    st.write("No image is uploaded yet!")

elif source_radio == settings.VIDEO:
  helper.play_stored_video(confidence, model)

else:
    st.error("Please select a valid source type!")

Overwriting app.py


In [33]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0

npx: installed 22 in 2.837s
your url is: https://five-streets-grin.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.56.243:8501


0: 448x640 2 helmets, 315.5ms
Speed: 7.0ms preprocess, 315.5ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 helmets, 236.5ms
Speed: 7.2ms preprocess, 236.5ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)
  Stopping...
Exception in callback _HandlerDelegate.execute.<locals>.<lambda>(<Task cancell.../web.py:1742>>) at /usr/local/lib/python3.10/dist-packages/tornado/web.py:2434
handle: <Handle _HandlerDelegate.execute.<locals>.<lambda>(<Task cancell.../web.py:1742>>) at /usr/local/lib/python3.10/dist-packages/tornado/web.py:2434>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tornado/web.py", line 1786, in _exe